<a href="https://colab.research.google.com/github/tanlull/Botnoi-ML/blob/main/Botnoi_Image_Classification_pipeline_%E0%B8%AB%E0%B8%A1%E0%B8%B2%E0%B9%81%E0%B8%A1%E0%B8%A7%E0%B8%A5%E0%B8%B4%E0%B8%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning

##Pipeline
1. get data
2. extract feature
3. train model
4. test model
5. deploy model

In [3]:
!pip uninstall keras -y
!pip uninstall tensorflow -y
!pip install keras==2.4.3 
!pip install tensorflow==2.4.3 
!pip install botnoi==0.2.1 

Found existing installation: keras 2.6.0
Uninstalling keras-2.6.0:
  Successfully uninstalled keras-2.6.0
Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Successfully uninstalled tensorflow-2.6.0
     |████████████████████████████████| 394.5 MB 378 bytes/s 
     |████████████████████████████████| 3.8 MB 29.2 MB/s 
     |████████████████████████████████| 462 kB 48.9 MB/s 
     |████████████████████████████████| 2.9 MB 52.9 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.39.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
 

In [4]:
import botnoi as bn
from botnoi import scrape as sc
from botnoi import cv
import os

In [5]:
def extractimagefeat(query):
  #create folder
  foldername = 'images/'+query
  isdir = os.path.isdir(foldername) 
  #check if folder exist
  if not isdir:
    #create directory
    os.makedirs(foldername)
  #get images from google search
  imglist = sc.get_image_urls(query,30) #ถ้าเรามีรูปภาพของตัวเอง มาแก้ตรงนี้คือใส่ file ของเราใน folder , 30 คือจำนวนรูป
  i = 1
  for img in imglist[0:30]:
    #extract image features from each images and save to files
    try:
      print(i)
      #create image path
      savepath = foldername + '/' + str(i)+'.p'
      a = cv.image(img)
      a.getresnet50()
      #a.getmobilenet()
      #a.getmobilenet()
      a.save(savepath)
      i = i + 1
    except:
      pass
  return 'complete'


In [6]:
import glob
import pandas as pd
import pickle
def createdataset(clsList):
  #imgfolder = glob.glob('images/*')
  dataset = []
  for cls in clsList:
    clsset = pd.DataFrame()
    pList = glob.glob('images/'+cls+'/*')
    featvec = []
    for p in pList:
      dat = pickle.load(open(p,'rb'))
      featvec.append(dat.resnet50)
    clsset['feature'] = featvec
    cls = cls.split('/')[-1]
    clsset['label'] = cls
    dataset.append(clsset)
  return pd.concat(dataset,axis=0)

In [7]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.svm import LinearSVC
def trainmodel(dataset,modfile=''):
  trainfeat, testfeat, trainlabel, testlabel = train_test_split(dataset['feature'], dataset['label'], test_size=0.20, random_state=0)
  clf = LinearSVC()
  mod = clf.fit(np.vstack(trainfeat.values),trainlabel.values)
  res = mod.predict(np.vstack(testfeat.values))
  if modfile!='':
    pickle.dump(mod,open(modfile,'wb'))
  acc = sum(res == testlabel)/len(res)
  return mod,acc

In [8]:
def createclassifier(clsList,modfile):
  for c in clsList:
    extractimagefeat(c)
  dset = createdataset(clsList)
  return trainmodel(dset,modfile)

In [9]:
#createclassifier(['หมา','แมว','ลิง'],'หมาแมวลิง.mod')
createclassifier(['ลุงตู่','ลุงป้อม','หมอหนู'],'ลุงตู่ลุงป้อมหมอหนู.mod')

complete: 0.00%
completed
1
94773248/94765736 [==============================] - 1s 0us/step
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
26
27
28
29
complete: 0.00%
completed
1
2
3
4
5
6
7
7
8
9
10
11
12
13
14
15
15
16
17
17
18
19
20
21
22
23
24
25
26
27
complete: 0.00%
completed
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
19
20
21
22
23
24
25
26
27
28
29


(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
           intercept_scaling=1, loss='squared_hinge', max_iter=1000,
           multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
           verbose=0), 0.6470588235294118)

In [10]:
# output function
modFile = 'ลุงตู่ลุงป้อมหมอหนู.mod'
mod = pickle.load(open(modFile,'rb'))

In [11]:
def predicting(imgurl):
  a = cv.image(imgurl)
  feat = a.getresnet50()
  res = mod.predict([feat])
  return res

In [12]:
a=predicting('https://www.thaipost.net/main/uploads/photos/big/20201222/image_big_5fe1d1a9624d7.jpg')
# a=predictimg('https://f.ptcdn.info/810/034/000/1440553775-IMG5594-o.jpg')
a

array(['ลุงตู่'], dtype=object)

In [13]:
def getthumbdisplay(imgList):
  res = ''
  for img in imgList:
    res = res + gethtmlimage(img)
  return res


In [14]:
def gethtmlimage(imgurl):
  htmlTag = '''
  <a target="_blank" href="%s">
    <img src="%s" alt="Forest">
  </a>
  '''%(imgurl,imgurl)
  return htmlTag


In [15]:
import IPython
from google.colab import output
def displayimagelist(imgList):
  htmlimage = getthumbdisplay(imgList)
  html = '''<style>
  img {
    border: 1px solid #ddd; /* Gray border */
    border-radius: 4px;  /* Rounded border */
    padding: 5px; /* Some padding */
    width: 150px; /* Set a small width */
  }

  /* Add a hover effect (blue shadow) */
  img:hover {
    box-shadow: 0 0 2px 1px rgba(0, 140, 186, 0.5);
  }
  </style>
  <body>

  %s

  </body>'''%htmlimage
  display(IPython.display.HTML(html))

In [16]:
imglist = sc.get_image_urls('หมา',30)
displayimagelist(imglist)

complete: 0.00%
completed
